<a href="https://colab.research.google.com/github/kumarashutosh22/Kaggle/blob/master/titanic.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Kaggle API

In [1]:
! mkdir ~/.kaggle

In [2]:
!cp kaggle.json ~/.kaggle/

In [3]:
!chmod 600 ~/.kaggle/kaggle.json

In [4]:
!kaggle competitions download -c titanic

  0% 0.00/28.0k [00:00<?, ?B/s]
100% 28.0k/28.0k [00:00<00:00, 23.6MB/s]
  0% 0.00/59.8k [00:00<?, ?B/s]
100% 59.8k/59.8k [00:00<00:00, 18.9MB/s]
  0% 0.00/3.18k [00:00<?, ?B/s]
100% 3.18k/3.18k [00:00<00:00, 3.22MB/s]


In [6]:
# kaggle datasets download -d landlord/handwriting-recognition

# Datasets and Libraries

In [7]:
import numpy as np
import pandas as pd

In [8]:
train = pd.read_csv('train.csv')
Xts = pd.read_csv('test.csv')
gs = pd.read_csv('gender_submission.csv')

# Basic EDA

In [25]:
train.shape,Xts.shape,gs.shape

((891, 12), (418, 11), (418, 2))

In [26]:
train.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [27]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  891 non-null    int64  
 1   Survived     891 non-null    int64  
 2   Pclass       891 non-null    int64  
 3   Name         891 non-null    object 
 4   Sex          891 non-null    object 
 5   Age          714 non-null    float64
 6   SibSp        891 non-null    int64  
 7   Parch        891 non-null    int64  
 8   Ticket       891 non-null    object 
 9   Fare         891 non-null    float64
 10  Cabin        204 non-null    object 
 11  Embarked     889 non-null    object 
dtypes: float64(2), int64(5), object(5)
memory usage: 83.7+ KB


In [28]:
# creating a dataframe to check the titles of all the passengers
title = []
for name in train.Name:
  title.append(name.split(',')[1].split('.')[0][1:])

pd.DataFrame(title,columns=['Title']).Title.value_counts()

Mr              517
Miss            182
Mrs             125
Master           40
Dr                7
Rev               6
Col               2
Major             2
Mlle              2
Don               1
Capt              1
Jonkheer          1
the Countess      1
Sir               1
Lady              1
Mme               1
Ms                1
Name: Title, dtype: int64

In [29]:
cab = []
for cabin in train.Cabin.dropna():
  cab.append(cabin[0])

pd.DataFrame(cab,columns=['Cabin']).Cabin.value_counts()

C    59
B    47
D    33
E    32
A    15
F    13
G     4
T     1
Name: Cabin, dtype: int64

In [30]:
train.isna().sum()

PassengerId      0
Survived         0
Pclass           0
Name             0
Sex              0
Age            177
SibSp            0
Parch            0
Ticket           0
Fare             0
Cabin          687
Embarked         2
dtype: int64

In [31]:
Xts.isna().sum()

PassengerId      0
Pclass           0
Name             0
Sex              0
Age             86
SibSp            0
Parch            0
Ticket           0
Fare             1
Cabin          327
Embarked         0
dtype: int64

# Data Cleaning

In [32]:
Xtr = train.drop(['Survived'],1)
ytr = train.Survived

In [33]:
X = pd.concat([Xtr,Xts],ignore_index=True)

In [34]:
X.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1309 entries, 0 to 1308
Data columns (total 11 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  1309 non-null   int64  
 1   Pclass       1309 non-null   int64  
 2   Name         1309 non-null   object 
 3   Sex          1309 non-null   object 
 4   Age          1046 non-null   float64
 5   SibSp        1309 non-null   int64  
 6   Parch        1309 non-null   int64  
 7   Ticket       1309 non-null   object 
 8   Fare         1308 non-null   float64
 9   Cabin        295 non-null    object 
 10  Embarked     1307 non-null   object 
dtypes: float64(2), int64(4), object(5)
memory usage: 112.6+ KB


In [35]:
# Encoding the Cabin column
for index in X.Cabin.dropna().index:
  X.loc[index,'Cabin'] = X.loc[index,'Cabin'][0]

X.Cabin = X.Cabin.map({'A':0,'B':1,'C':3,'D':4,'E':5,'F':6,'G':7,'T':19})

In [36]:
# Encoding sex column
X.Sex = X.Sex.map({'male':0,'female':1})

In [37]:
# encoding embarked column
X.Embarked = X.Embarked.map({'S':0,'C':1,'Q':2})

In [38]:
X.drop(['Name','Ticket'],axis=1,inplace=True)

In [39]:
from sklearn.impute import KNNImputer

imputer = KNNImputer(n_neighbors=5)
Xf = pd.DataFrame(imputer.fit_transform(X),columns=X.columns)

In [40]:
Xftr = Xf[:891]
Xfts = Xf[891:]

In [41]:
Xts.shape,Xfts.shape

((418, 11), (418, 9))

# Regression

In [18]:
from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score

In [42]:
xgb = XGBClassifier(colsample_bytree=0.4,
                 gamma=0,                 
                 learning_rate=0.07,
                 max_depth=3,
                 min_child_weight=1.5,
                 n_estimators=10000,                                                                    
                 reg_alpha=0.75,
                 reg_lambda=0.45,
                 subsample=0.6,
                 seed=42).fit(Xftr,ytr)

In [43]:
ypr = xgb.predict(Xfts)

In [44]:
accuracy_score(gs.Survived,ypr)

1.0

In [45]:
gs.Survived = ypr

In [46]:
gs.to_csv('submission.csv',index=False)

In [48]:
!kaggle competitions submit -c titanic -f submission.csv -m "Message"

100% 2.77k/2.77k [00:03<00:00, 805B/s]
Successfully submitted to Titanic: Machine Learning from Disaster